In [ ]:
import pandas as pd
from collections import Counter
import json
import torch
import torch.nn as nn
from torch.utils.data import Dataset
import torch.utils.data
import math
import torch.nn.functional as F

USE_CUDA = torch.cuda.is_available()
device = torch.device("cuda" if USE_CUDA else "cpu")

In [ ]:
df = pd.read_csv("customized-mutants.csv")
df

,projectId,bugId,methodName,mutantId,compositeId,className,lineNumber,testSignature,mutationOperatorGroup,mutationOperator,...,nestingLoop,nestingIf,maxNestingInSameMethod,nestingRatioTotal,nestingRatioLoop,nestingRatioIf,numMutantsInSameMethod,maxLineNumberInSameMethod,minLineNumberInSameMethod,lineRatio
0,Codec,18,NaN,10390,Codec-18/10390,org.apache.commons.codec.digest.PureJavaCrc32C,345,AvEAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA...,LVR,INT_LITERAL:INT_LITERAL,...,1,1,5,0.2,0.2,0.2,11584,939,29,0.347253
1,Codec,18,NaN,6773,Codec-18/6773,org.apache.commons.codec.digest.PureJavaCrc32,438,AvEAAAAEAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA...,LVR,INT_LITERAL:INT_LITERAL,...,1,1,5,0.2,0.2,0.2,11584,939,29,0.449451
2,Codec,18,NaN,12430,Codec-18/12430,org.apache.commons.codec.digest.PureJavaCrc32C,604,AvEAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA...,LVR,INT_LITERAL:INT_LITERAL,...,1,1,5,0.2,0.2,0.2,11584,939,29,0.631868
3,Codec,18,NaN,6726,Codec-18/6726,org.apache.commons.codec.digest.PureJavaCrc32,431,AvEAAAAEAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA...,LVR,INT_LITERAL:INT_LITERAL,...,1,1,5,0.2,0.2,0.2,11584,939,29,0.441758
4,Codec,18,NaN,12433,Codec-18/12433,org.apache.commons.codec.digest.PureJavaCrc32C,604,AvEAAAAAAAAAAAAAAAABAAAAAAAAAAAAAAAAAAAAAAAAAA...,LVR,INT_LITERAL:INT_LITERAL,...,1,1,5,0.2,0.2,0.2,11584,939,29,0.631868
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24818,Codec,18,write(int),1610,Codec-18/1610,org.apache.commons.codec.binary.BaseNCodecOutp...,67,AvEAAAAAACAAAAAAAAAAAAAAAAAACAAAAAAAAAAAAAAAAA...,LVR,INT_LITERAL:INT_LITERAL,...,1,1,1,1.0,1.0,1.0,8,68,67,0.000000
24819,Codec,18,write(int),1615,Codec-18/1615,org.apache.commons.codec.binary.BaseNCodecOutp...,68,AvEAAAAAACAAAAAAAAAAAAAAAAAACAAAAAAAAAAAAAAAAA...,LVR,INT_LITERAL:INT_LITERAL,...,1,1,1,1.0,1.0,1.0,8,68,67,1.000000
24820,Codec,18,write(int),1614,Codec-18/1614,org.apache.commons.codec.binary.BaseNCodecOutp...,68,AvEAAAAAACAAAAAAAAAAAAAAAAAACAAAAAAAAAAAAAAAAA...,LVR,INT_LITERAL:INT_LITERAL,...,1,1,1,1.0,1.0,1.0,8,68,67,1.000000
24821,Codec,18,write(int),1612,Codec-18/1612,org.apache.commons.codec.binary.BaseNCodecOutp...,68,AvEAAAAAACAAAAAAAAAAAAAAAAAACAAAAAAAAAAAAAAAAA...,LVR,INT_LITERAL:INT_LITERAL,...,1,1,1,1.0,1.0,1.0,8,68,67,1.000000


In [ ]:
df = df[df["pKillsDom"] >= 0.75] # 0.25, 0.5, 0.75

In [ ]:
dict_list = df.to_dict('records')
before_mutations = []
after_mutations = []
for dd in dict_list:
  mutationOperator = dd["mutationOperator"]
  ml = mutationOperator.split(":")
  before_mutations.append(ml[0])
  after_mutations .append(ml[1])
df["before_mutation"] = before_mutations
df["after_mutation"] = after_mutations

<ipython-input-221-67678cebeffa>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["before_mutation"] = before_mutations
<ipython-input-221-67678cebeffa>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["after_mutation"] = after_mutations


In [ ]:
pairs = []
dict_list = df.to_dict('records')
for dd in dict_list:
  qa_pairs = []
  if dd["methodName"] and not pd.isna(dd["methodName"]):
    methodName = dd["methodName"] 
  else:
    methodName = "Unknown"
  operation_before = dd["before_mutation"]
  mutationOperatorGroup = dd["mutationOperatorGroup"]
  operation_after = dd["after_mutation"]
  nestingLoop = dd["nestingLoop"]
  nestingIf = dd["nestingIf"]
  parentStmtContextDetailed = dd["parentStmtContextDetailed"]
  question = [methodName, operation_before, f"{nestingLoop}", f"{nestingIf}",f"{parentStmtContextDetailed}"]
  answer = [dd["after_mutation"]]
  qa_pairs.append(question)
  qa_pairs.append(answer)
  pairs.append(qa_pairs)

In [ ]:
print(pairs[0][0])
print(pairs[0][1])

['Unknown', 'INT_LITERAL', '1', '1', 'VARIABLE']
['INT_LITERAL']


In [ ]:
word_freq = Counter()
for pair in pairs:
    word_freq.update(pair[0])
    word_freq.update(pair[1])

In [ ]:
min_word_freq = 5
words = [w for w in word_freq.keys() if word_freq[w] > min_word_freq]
word_map = {k: v + 1 for v, k in enumerate(words)}
word_map['<unk>'] = len(word_map) + 1
word_map['<start>'] = len(word_map) + 1
word_map['<end>'] = len(word_map) + 1
word_map['<pad>'] = 0

In [ ]:
print("Total words are: {}".format(len(word_map)))

Total words are: 198


In [ ]:
MAX_LEN = 30

In [ ]:
def encode_request(words, word_map):
    enc_c = [word_map.get(word, word_map['<unk>']) for word in words] + [word_map['<pad>']] * (MAX_LEN - len(words))
    return enc_c

In [ ]:
def encode_answer(words, word_map):
    enc_c = [word_map['<start>']] + [word_map.get(word, word_map['<unk>']) for word in words] + \
    [word_map['<end>']] + [word_map['<pad>']] * (MAX_LEN - len(words))
    return enc_c

In [ ]:
pairs_encoded = []
for pair in pairs:
    qus = encode_request(pair[0], word_map)
    ans = encode_answer(pair[1], word_map)
    pairs_encoded.append([qus, ans])

In [ ]:
class Dataset(Dataset):

    def __init__(self):
        self.pairs = pairs_encoded
        self.dataset_size = len(self.pairs)

    def __getitem__(self, i):
        question = torch.LongTensor(self.pairs[i][0])
        reply = torch.LongTensor(self.pairs[i][1])
        return question, reply

    def __len__(self):
        return self.dataset_size

In [ ]:
train_loader = torch.utils.data.DataLoader(Dataset(),
                                           batch_size = 100, 
                                           shuffle=True, 
                                           pin_memory=True)

In [ ]:
def create_masks(question, reply_input, reply_target):
    
    def subsequent_mask(size):
        mask = torch.triu(torch.ones(size, size)).transpose(0, 1).type(dtype=torch.uint8)
        return mask.unsqueeze(0)
    
    question_mask = question!=0
    question_mask = question_mask.to(device)
    question_mask = question_mask.unsqueeze(1).unsqueeze(1)         # (batch_size, 1, 1, max_words)
     
    reply_input_mask = reply_input!=0
    reply_input_mask = reply_input_mask.unsqueeze(1)  # (batch_size, 1, max_words)
    reply_input_mask = reply_input_mask & subsequent_mask(reply_input.size(-1)).type_as(reply_input_mask.data) 
    reply_input_mask = reply_input_mask.unsqueeze(1) # (batch_size, 1, max_words, max_words)
    reply_target_mask = reply_target!=0              # (batch_size, max_words)
    
    return question_mask, reply_input_mask, reply_target_mask

In [ ]:
class Embeddings(nn.Module):
    def __init__(self, vocab_size, d_model, max_len = 50):
        super(Embeddings, self).__init__()
        self.d_model = d_model
        self.dropout = nn.Dropout(0.1)
        self.embed = nn.Embedding(vocab_size, d_model)
        self.pe = self.positinal_encoding(max_len, self.d_model)
        self.dropout = nn.Dropout(0.1)
        
    def positinal_encoding(self, max_len, d_model):
        pe = torch.zeros(max_len, d_model).to(device)
        position = torch.arange(max_len).unsqueeze(1)
        dim_pair = torch.arange(0, d_model, 2)
        div_term = torch.exp(dim_pair * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        return pe
        
    def forward(self, encoded_words):
        embedding = self.embed(encoded_words) * math.sqrt(self.d_model)
        embedding += self.pe[:, :embedding.size(1)]
        embedding = self.dropout(embedding)
        return embedding

In [ ]:
class MultiHeadAttention(nn.Module):
    
    def __init__(self, heads, d_model):
        
        super(MultiHeadAttention, self).__init__()
        assert d_model % heads == 0
        self.d_k = d_model // heads
        self.heads = heads
        self.dropout = nn.Dropout(0.1)
        self.query = nn.Linear(d_model, d_model)
        self.key = nn.Linear(d_model, d_model)
        self.value = nn.Linear(d_model, d_model)
        self.concat = nn.Linear(d_model, d_model)
        
    def forward(self, query, key, value, mask):
        """
        query, key, value of shape: (batch_size, max_len, 512)
        mask of shape: (batch_size, 1, 1, max_words)
        """
        # (batch_size, max_len, 512)
        query = self.query(query)
        key = self.key(key)        
        value = self.value(value)   
        
        # (batch_size, max_len, 512) --> (batch_size, max_len, h, d_k) --> (batch_size, h, max_len, d_k)
        query = query.view(query.shape[0], -1, self.heads, self.d_k).permute(0, 2, 1, 3)   
        key = key.view(key.shape[0], -1, self.heads, self.d_k).permute(0, 2, 1, 3)  
        value = value.view(value.shape[0], -1, self.heads, self.d_k).permute(0, 2, 1, 3)  
        
        # (batch_size, h, max_len, d_k) matmul (batch_size, h, d_k, max_len) --> (batch_size, h, max_len, max_len)
        scores = torch.matmul(query, key.permute(0,1,3,2)) / math.sqrt(query.size(-1))
        scores = scores.masked_fill(mask == 0, -1e9)    # (batch_size, h, max_len, max_len)
        weights = F.softmax(scores, dim = -1)           # (batch_size, h, max_len, max_len)
        weights = self.dropout(weights)
        # (batch_size, h, max_len, max_len) matmul (batch_size, h, max_len, d_k) --> (batch_size, h, max_len, d_k)
        context = torch.matmul(weights, value)
        # (batch_size, h, max_len, d_k) --> (batch_size, max_len, h, d_k) --> (batch_size, max_len, h * d_k)
        context = context.permute(0,2,1,3).contiguous().view(context.shape[0], -1, self.heads * self.d_k)
        # (batch_size, max_len, h * d_k)
        interacted = self.concat(context)
        return interacted 

In [ ]:
class FeedForward(nn.Module):

    def __init__(self, d_model, middle_dim = 2048):
        super(FeedForward, self).__init__()
        
        self.fc1 = nn.Linear(d_model, middle_dim)
        self.fc2 = nn.Linear(middle_dim, d_model)
        self.dropout = nn.Dropout(0.1)

    def forward(self, x):
        out = F.relu(self.fc1(x))
        out = self.fc2(self.dropout(out))
        return out

In [ ]:
class EncoderLayer(nn.Module):

    def __init__(self, d_model, heads):
        super(EncoderLayer, self).__init__()
        self.layernorm = nn.LayerNorm(d_model)
        self.self_multihead = MultiHeadAttention(heads, d_model)
        self.feed_forward = FeedForward(d_model)
        self.dropout = nn.Dropout(0.1)

    def forward(self, embeddings, mask):
        interacted = self.dropout(self.self_multihead(embeddings, embeddings, embeddings, mask))
        interacted = self.layernorm(interacted + embeddings)
        feed_forward_out = self.dropout(self.feed_forward(interacted))
        encoded = self.layernorm(feed_forward_out + interacted)
        return encoded

In [ ]:
class DecoderLayer(nn.Module):
    
    def __init__(self, d_model, heads):
        super(DecoderLayer, self).__init__()
        self.layernorm = nn.LayerNorm(d_model)
        self.self_multihead = MultiHeadAttention(heads, d_model)
        self.src_multihead = MultiHeadAttention(heads, d_model)
        self.feed_forward = FeedForward(d_model)
        self.dropout = nn.Dropout(0.1)
        
    def forward(self, embeddings, encoded, src_mask, target_mask):
        query = self.dropout(self.self_multihead(embeddings, embeddings, embeddings, target_mask))
        query = self.layernorm(query + embeddings)
        interacted = self.dropout(self.src_multihead(query, encoded, encoded, src_mask))
        interacted = self.layernorm(interacted + query)
        feed_forward_out = self.dropout(self.feed_forward(interacted))
        decoded = self.layernorm(feed_forward_out + interacted)
        return decoded

In [ ]:
class Transformer(nn.Module):
    
    def __init__(self, d_model, heads, num_layers, word_map):
        super(Transformer, self).__init__()
        
        self.d_model = d_model
        self.vocab_size = len(word_map)
        self.embed = Embeddings(self.vocab_size, d_model)
        self.encoder = nn.ModuleList([EncoderLayer(d_model, heads) for _ in range(num_layers)])
        self.decoder = nn.ModuleList([DecoderLayer(d_model, heads) for _ in range(num_layers)])
        self.logit = nn.Linear(d_model, self.vocab_size)
        
    def encode(self, src_words, src_mask):
        src_embeddings = self.embed(src_words)
        for layer in self.encoder:
            src_embeddings = layer(src_embeddings, src_mask)
        return src_embeddings
    
    def decode(self, target_words, target_mask, src_embeddings, src_mask):
        tgt_embeddings = self.embed(target_words)
        for layer in self.decoder:
            tgt_embeddings = layer(tgt_embeddings, src_embeddings, src_mask, target_mask)
        return tgt_embeddings
        
    def forward(self, src_words, src_mask, target_words, target_mask):
        encoded = self.encode(src_words, src_mask)
        decoded = self.decode(target_words, target_mask, encoded, src_mask)
        out = F.log_softmax(self.logit(decoded), dim = 2)
        return out

In [ ]:
class AdamWarmup:
    
    def __init__(self, model_size, warmup_steps, optimizer):
        
        self.model_size = model_size
        self.warmup_steps = warmup_steps
        self.optimizer = optimizer
        self.current_step = 0
        self.lr = 0
        
    def get_lr(self):
        return self.model_size ** (-0.5) * min(self.current_step ** (-0.5), self.current_step * self.warmup_steps ** (-1.5))
        
    def step(self):
        # Increment the number of steps each time we call the step function
        self.current_step += 1
        lr = self.get_lr()
        for param_group in self.optimizer.param_groups:
            param_group['lr'] = lr
        # update the learning rate
        self.lr = lr
        self.optimizer.step()    

In [ ]:
class LossWithLS(nn.Module):

    def __init__(self, size, smooth):
        super(LossWithLS, self).__init__()
        self.criterion = nn.KLDivLoss(size_average=False, reduce=False)
        self.confidence = 1.0 - smooth
        self.smooth = smooth
        self.size = size
        
    def forward(self, prediction, target, mask):
        """
        prediction of shape: (batch_size, max_words, vocab_size)
        target and mask of shape: (batch_size, max_words)
        """
        prediction = prediction.view(-1, prediction.size(-1))   # (batch_size * max_words, vocab_size)
        target = target.contiguous().view(-1)   # (batch_size * max_words)
        mask = mask.float()
        mask = mask.view(-1)       # (batch_size * max_words)
        labels = prediction.data.clone()
        labels.fill_(self.smooth / (self.size - 1))
        labels.scatter_(1, target.data.unsqueeze(1), self.confidence)
        loss = self.criterion(prediction, labels)    # (batch_size * max_words, vocab_size)
        loss = (loss.sum(1) * mask).sum() / mask.sum()
        return loss

In [ ]:
d_model = 512
heads = 2 # 2, 4, 8
num_layers = 3
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
epochs = 200    
transformer = Transformer(d_model = d_model, heads = heads, num_layers = num_layers, word_map = word_map)
transformer = transformer.to(device)
adam_optimizer = torch.optim.Adam(transformer.parameters(), lr=0, betas=(0.9, 0.98), eps=1e-9)
transformer_optimizer = AdamWarmup(model_size = d_model, warmup_steps = 4000, optimizer = adam_optimizer)
criterion = LossWithLS(len(word_map), 0.1)

In [ ]:
def train(train_loader, transformer, criterion, epoch):
    
    transformer.train()
    sum_loss = 0
    count = 0

    for i, (question, reply) in enumerate(train_loader):
        
        samples = question.shape[0]

        # Move to device
        question = question.to(device)
        reply = reply.to(device)

        # Prepare Target Data
        reply_input = reply[:, :-1]
        reply_target = reply[:, 1:]

        # Create mask and add dimensions
        question_mask, reply_input_mask, reply_target_mask = create_masks(question, reply_input, reply_target)

        # Get the transformer outputs
        out = transformer(question, question_mask, reply_input, reply_input_mask)

        # Compute the loss
        loss = criterion(out, reply_target, reply_target_mask)
        
        # Backprop
        transformer_optimizer.optimizer.zero_grad()
        loss.backward()
        transformer_optimizer.step()
        
        sum_loss += loss.item() * samples
        count += samples
        
        if i % 100 == 0:
            print("Epoch [{}][{}/{}]\tLoss: {:.3f}".format(epoch, i, len(train_loader), sum_loss/count))

In [ ]:
def evaluate(transformer, question, question_mask, max_len, word_map):
    """
    Performs Greedy Decoding with a batch size of 1
    """
    rev_word_map = {v: k for k, v in word_map.items()}
    transformer.eval()
    start_token = word_map['<start>']
    encoded = transformer.encode(question, question_mask)
    words = torch.LongTensor([[start_token]]).to(device)
    
    for step in range(max_len - 1):
        size = words.shape[1]
        target_mask = torch.triu(torch.ones(size, size)).transpose(0, 1).type(dtype=torch.uint8)
        target_mask = target_mask.to(device).unsqueeze(0).unsqueeze(0)
        decoded = transformer.decode(words, target_mask, encoded, question_mask)
        predictions = transformer.logit(decoded[:, -1])
        _, next_word = torch.max(predictions, dim = 1)
        next_word = next_word.item()
        if next_word == word_map['<end>']:
            break
        words = torch.cat([words, torch.LongTensor([[next_word]]).to(device)], dim = 1)   # (1,step+2)
        
    # Construct Sentence
    if words.dim() == 2:
        words = words.squeeze(0)
        words = words.tolist()
        
    sen_idx = [w for w in words if w not in {word_map['<start>']}]
    sentence = ' '.join([rev_word_map[sen_idx[k]] for k in range(len(sen_idx))])
    
    return sentence

In [ ]:
for epoch in range(epochs):
    
    train(train_loader, transformer, criterion, epoch)
    
    state = {'epoch': epoch, 'transformer': transformer, 'transformer_optimizer': transformer_optimizer}
    # torch.save(state, 'checkpoint_' + str(epoch) + '.pth.tar')

Epoch [0][0/129]	Loss: 4.418
Epoch [0][100/129]	Loss: 1.888
Epoch [1][0/129]	Loss: 0.463
Epoch [1][100/129]	Loss: 0.427
Epoch [2][0/129]	Loss: 0.296
Epoch [2][100/129]	Loss: 0.264
Epoch [3][0/129]	Loss: 0.196
Epoch [3][100/129]	Loss: 0.249
Epoch [4][0/129]	Loss: 0.240
Epoch [4][100/129]	Loss: 0.245
Epoch [5][0/129]	Loss: 0.236
Epoch [5][100/129]	Loss: 0.245
Epoch [6][0/129]	Loss: 0.201
Epoch [6][100/129]	Loss: 0.242
Epoch [7][0/129]	Loss: 0.312
Epoch [7][100/129]	Loss: 0.237
Epoch [8][0/129]	Loss: 0.258
Epoch [8][100/129]	Loss: 0.239
Epoch [9][0/129]	Loss: 0.211
Epoch [9][100/129]	Loss: 0.242
Epoch [10][0/129]	Loss: 0.200
Epoch [10][100/129]	Loss: 0.237
Epoch [11][0/129]	Loss: 0.241
Epoch [11][100/129]	Loss: 0.236
Epoch [12][0/129]	Loss: 0.217
Epoch [12][100/129]	Loss: 0.238
Epoch [13][0/129]	Loss: 0.241
Epoch [13][100/129]	Loss: 0.235
Epoch [14][0/129]	Loss: 0.236
Epoch [14][100/129]	Loss: 0.236
Epoch [15][0/129]	Loss: 0.243


KeyboardInterrupt: ignored

In [ ]:
torch.save(model.state_dict(), "transformer.tar")

In [ ]:
with open('WORDMAP_corpus.json', 'w') as j:
    json.dump(word_map, j)